In [1]:
using LinearAlgebra
import Base.*, Base.\, Base.setindex!, Base.getindex, Base.size, Base.firstindex, Base.lastindex
# using Setfield

In [2]:
mutable struct RazprsenaMatrika{T} <: AbstractArray{T, 2}
    I::Matrix{Int}
    V::Matrix{T}
    n::Int
    m::Int
end

In [3]:
function size(M::RazprsenaMatrika)
    return (M.n, M.m)
end

function getindex(M::RazprsenaMatrika, I::Vararg{Int, 2})
    j = findfirst(isequal(I[2]), M.I[I[1], :])
    if j == nothing
        return 0 
    end
    return M.V[I[1], j]
end

function setindex!(M::RazprsenaMatrika, v, I::Vararg{Int, 2})
    i = I[1]
    j = findfirst(isequal(I[2]), M.I[i, :])
    
    if v == 0 && j == nothing
        return
    elseif v == 0
        # spraznimo index
        M.I[i, j:end-1] = M.I[i, j+1:end]
        M.I[i, end] = 0
        M.V[i, j:end-1] = M.V[i, j+1:end]
        M.V[i, end] = 0
        return
    elseif j == nothing
        # add index
        # check if space
        jx = findfirst(isequal(0), M.I[i, :])
        if jx == nothing
            # ni plac --> add col
            ri = zeros(M.n)
            ri[i] = I[2]
            M.I = hcat(M.I, ri)
            rv = zeros(M.n)
            rv[i] = v
            M.V = hcat(M.V, rv)
        else
            # je plac
            M.I[i, jx] = I[2]
            M.V[i, jx] = v
        end
    else
        M.V[i, j] = v
    end
end

function firstindex(M::RazprsenaMatrika)
    # for i=1:M.n
    #     j = M.I[i, 1]
    #     if j > 0
    #         return M.n * (j - 1) + i 
    #     end
    # end
    # @warn "Matrika ne vsebuje neničelnih elementov"
    return 1    
end

"""
    firstindex(M, d)

Vrne indeks prvega neničelnega elementa v vrstici d
"""
function firstindex(M::RazprsenaMatrika, d)
    # j = M.I[d, 1]
    # if j == 0
    #     @warn "Vrstica ne vsebuje neničelnega elementa"
    #     return 1
    # end
    # return j
    return 1
end

function lastindex(M::RazprsenaMatrika)
    # for i = M.n:-1:1
    #     j = findlast(x -> x > 0, M.I[i, :])
    #     if j != nothing
    #         return M.n * (M.I[i, j] - 1)  + i
    #     end
    # end
    # @warn "Matrika ne vsebuje neničelnih elementov"
    return M.n*M.m
end

"""
    lastindex(M, d)

Vrne indeks zadnjega neničelnega elementa v vrstici d
"""
function lastindex(M::RazprsenaMatrika, d)
    # j = findlast(x -> x > 0, M.I[d, :])
    # if j == nothing
    #     @warn "Vrstica ne vsebuje neničelnega elementa"
    #     return M.n
    # end
    # return M.I[d, j]  
    return M.m
end

Base.lastindex

In [4]:
function *(M::RazprsenaMatrika, v::Vector)
    x = zeros(M.n)
    for i=1:M.n
        ids = I[i, :].>0
        x[i] = dot(V[i, ids], v[I[i, ids]])
    end
    return x
end

* (generic function with 365 methods)

In [5]:
I = [4 0; 0 0; 1 3; 4 0; 0 0]
V = [1.0 0; 0 0; 3 4; 2 0; 0 0]
A = RazprsenaMatrika(I, V, 5, 5)

5×5 RazprsenaMatrika{Float64}:
 0    0  0    1.0  0
 0    0  0    0    0
 3.0  0  4.0  0    0
 0    0  0    2.0  0
 0    0  0    0    0

In [6]:
B = copy(A)
B

5×5 Matrix{Float64}:
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0
 3.0  0.0  4.0  0.0  0.0
 0.0  0.0  0.0  2.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [7]:
x = [1, 2, 34, 23, 6]
x' * (A * x)

5807.0

In [41]:
function conj_grad(M::RazprsenaMatrika, b::Vector, eps=1e-10, maxitr=100)
    x = zeros(M.n)
    r = b - M*x
    dotr = dot(r, r) # old r dot product
    if dotr < eps
        return (x, k)
    end
    p = r
    k = 1
    while true
        display(dotr)
        mp = M * p
        alpha = dotr / (p' * mp)
        x += alpha * p
        r -= alpha * mp
        dotr1 = dot(r, r)
        if dotr1 < eps
            break
        end
        beta = dotr1 / dotr
        dotr = dotr1 # update
        p = r + beta * p
        k += 1
        if k>maxitr
            break
        end
    end
    return (x, k)
end


conj_grad (generic function with 3 methods)

In [9]:
I = [4 0; 0 0; 1 3; 4 0; 0 0]
V = [1.0 0; 0 0; 3 4; 2 0; 0 0]
A = RazprsenaMatrika(I, V, 5, 5)
A[1, 1] = 12
A[2, 2] = 2
A[5, 5] = 8
A

5×5 RazprsenaMatrika{Float64}:
 12.0  0    0    1.0  0
  0    2.0  0    0    0
  3.0  0    4.0  0    0
  0    0    0    2.0  0
  0    0    0    0    8.0

In [11]:
A[1, 2] = 123

123

In [18]:
a = A'*A

5×5 Matrix{Float64}:
  153.0   1476.0  12.0   12.0   0.0
 1476.0  15133.0   0.0  123.0   0.0
   12.0      0.0  16.0    0.0   0.0
   12.0    123.0   0.0    5.0   0.0
    0.0      0.0   0.0    0.0  64.0

In [19]:
isposdef(a)

true

In [20]:
n = 3
m = 4
A = RazprsenaMatrika(zeros(Int,n, 1), zeros(n, 1), n, m)

3×4 RazprsenaMatrika{Float64}:
 0  0  0  0
 0  0  0  0
 0  0  0  0

In [21]:
function matrika(M::Matrix)
    n, m = size(M)
    A = RazprsenaMatrika(zeros(Int,n, 1), zeros(n, 1), n, m)
    for i=1:n
        for j=1:m
            if M[i, j] != 0
            #     display(i)
            #     display(j)
            #     display(M[i, j])
                A[i, j] = M[i, j]
            end
        end
    end
    return A
end

matrika (generic function with 1 method)

In [42]:
ra = matrika(a)

5×5 RazprsenaMatrika{Float64}:
  153.0   1476.0  12.0   12.0   0
 1476.0  15133.0   0    123.0   0
   12.0      0    16.0    0     0
   12.0    123.0   0      5.0   0
    0        0     0      0    64.0

In [43]:
b = ones(ra.n)*12

5-element Vector{Float64}:
 12.0
 12.0
 12.0
 12.0
 12.0

In [44]:
x,i = conj_grad(ra, b)

720.0

299.53125

84.02507716049382

4.114148355867818

0.5562807189300402

...

1.37874208959857e-8

5.115934825077414e-9

2.304393400845449e-9

9.378906686114988e-10

([0.5000003631326823, 6.000000275586383, 2.6250009778817396, 6.000000275586383, 1.4999999201986092], 18)

In [21]:
A*x

5-element Vector{Float64}:
  0.9999998901552312
  2.000000049656173
 34.000000312368265
 23.000000571045998
  5.999999506026069

In [23]:
norm(A*x - b, Inf) 

5.710459980434734e-7

In [6]:
import SparseArrays.sparse

In [11]:
s = sparse(B)

5×5 SparseArrays.SparseMatrixCSC{Float64, Int64} with 4 stored entries:
  ⋅    ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅ 
 3.0   ⋅   4.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅   2.0   ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅ 

In [15]:
lastindex(s, 1)

5

In [ ]:
# example
